In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
from sklearn import preprocessing

# ! python -m spacy download en_core_web_md #you will need to install this on first load
import spacy
from spacy.lang.en import English
from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()
from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow
tf.disable_eager_execution()

import json
import pickle

In [ ]:
####Load News Articles

In [ ]:
sentencesList = []
with open('./News Data/News_Category_Dataset_v2.json') as f:
    for jsonObj in f:
        jsonDict = json.loads(jsonObj)
        sentencesList.append( (jsonDict["headline"], jsonDict["short_description"], jsonDict["date"]) )

# Sort by Date
sentencesList.sort(key = lambda element : element[2])

# Headline + Summary
headlinesWithSummary = []

# Only Headline
headlines = []

for sentence in sentencesList:
    # sentence[0] : Headline
    # sentence[1] : Short description

    headline = sentence[0].lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').strip()
    headlines.append(headline)
    
    text = sentence[0] + ". " + sentence[1]
    text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').strip()
    headlinesWithSummary.append(text)
    
# The number of Articles
print(len(headlinesWithSummary))

with open('./News Data/pre-processed_news_headline_list.pickle', 'wb') as f:
    pickle.dump(headlines, f)
with open('./News Data/pre-processed_news_headline_with_summary_list.pickle', 'wb') as f:
    pickle.dump(headlinesWithSummary, f)


In [ ]:
####Create Sentence Embeddings

In [ ]:
# Importing trained ELMo model ("https://tfhub.dev/google/elmo/3")
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [ ]:
# The number of articles processed in one session
batchSize = 400

# The number of sessions saved in one file 
# ( batchSize * batchNum articles stored in one file. )
batchNum = 100

# The number of sentence vector files
epoach = 5

# Using only Headline for sentence embedding
sentences = headlines

In [ ]:
# Creating Sentence Vector files
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    #for c in range(epoach):
    for c in range(1,epoach):
        batchEmbed = []
        for i in range(c*batchNum, (c+1)*(batchNum)):
            print("c:", c, "i:", i)
            embeddings = sess.run(embed(sentences[i*batchSize:(i+1)*batchSize],signature="default",as_dict=True)["default"])
            batchEmbed.append(embeddings)
        
        conc = batchEmbed[0]
        print(conc)
        for i in range(1,batchNum):
            conc = np.concatenate((conc, batchEmbed[i]), axis=0)
        print(conc.shape)
        
        with open('./Sentence Vector/SentenceVector_'+str(c)+'.pickle', 'wb') as f:
            pickle.dump(conc, f)
            